In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
import re


In [8]:
big_df = pd.read_csv('report_DP', sep=r' \| ', engine='python')
big_df

,algo,arch,scale,query,reln,limit,reps,eps,pre_eps,run_eps,nosy_ans,gsens,bias_ans,true_ans,nosy_ans_list,bias_ans_list,gsens_list,avg_time,time_list
0,TSensDP,tpch,0.01,q1,CUSTOMER,100,20,1.0,0.5,0.5,5.758217e+04,122.40,57703.90,60175.0,"[58430.743052637816, 57820.08176333646, 59421....","[57803.0, 57803.0, 60036.0, 59134.0, 58401.0, ...","[118, 118, 135, 124, 120, 105, 92, 131, 113, 1...",0.692514,"[0.696265459060669, 0.6604721546173096, 0.6665..."
1,TSensDP,tpch,0.01,q2,SUPPLIER,500,20,1.0,0.5,0.5,4.937317e+04,679.80,49451.45,60175.0,"[59565.115173782586, 61489.73065199166, 18880....","[60175.0, 60175.0, 20129.0, 60175.0, 60175.0, ...","[759, 880, 594, 680, 701, 1115, 651, 598, 620,...",0.553728,"[0.5810489654541016, 0.5946664810180664, 0.560..."
2,TSensDP,tpch,0.01,q3,CUSTOMER,10,20,1.0,0.5,0.5,2.303708e+03,18.45,2296.65,2333.0,"[2281.160710492658, 2366.059196160785, 2271.06...","[2262.0, 2333.0, 2284.0, 2262.0, 2333.0, 2333....","[10, 14, 11, 10, 27, 50, 33, 11, 14, 10, 13, 1...",23.063181,"[23.55450963973999, 22.874011993408203, 23.050..."
3,PrivateSQL,tpch,0.01,q1,CUSTOMER,None,20,1.0,0.5,0.5,5.945260e+04,254.85,59337.40,60175.0,"[60133.48120771961, 59402.26065834167, 60662.6...","[59560, 60175, 59560, 59440, 57571, 60175, 594...","[217, 256, 217, 210, 189, 238, 510, 189, 224, ...",0.291768,"[0.30391526222229004, 0.2895805835723877, 0.28..."
4,PrivateSQL,tpch,0.01,q2,SUPPLIER,None,20,1.0,0.5,0.5,5.187611e+02,867.80,0.00,60175.0,"[28.436287891413627, 431.2426133975302, 234.89...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[648, 664, 902, 715, 902, 747, 1280, 1344, 810...",0.230850,"[0.23453831672668457, 0.23037099838256836, 0.2..."
5,PrivateSQL,tpch,0.01,q3,CUSTOMER,None,20,1.0,0.5,0.5,-8.018122e+06,14242500.00,2266.90,2333.0,"[60413741.45147254, -23521161.476532884, 32318...","[2238, 2333, 2259, 2333, 2238, 2070, 2238, 228...","[13500000, 11200000, 9800000, 17600000, 945000...",0.546385,"[0.5631613731384277, 0.5541017055511475, 0.553..."


In [9]:
def num_symbol(num):
    return '+' if num > 0 else '' 

def beautify_float(f):
    return '%3.0f'%f

bf = beautify_float        

In [16]:
q_descp = {
    'q1': 'path join',
    'q2': 'acyclic join',
    'q3': 'cyclic join'
}

print('query', 'type', 'true answer', 'bias', 'sensitivity', 'error', 'time', 'bias', 'sensitivity', 'error', 'time', sep='\t')

for q in ['q1', 'q2', 'q3']:
    df_TSensDP = big_df.query('algo == "{algo}" & query == "{q}"'.format(algo='TSensDP', q=q))
    df_PrivateSQL = big_df.query('algo == "{algo}" & query == "{q}"'.format(algo='PrivateSQL', q=q))
    df_list = [df_TSensDP, df_PrivateSQL]
    
    cols = []
    for df in df_list:
        #print(q, df['algo_name'].tolist()[0])
        true_ans = int(df['true_ans'].tolist()[0])
        nosy_ans = df['nosy_ans'].tolist()[0]
        #gsens = df['gsens'].tolist()[0]
        reln = df['reln'].tolist()[0]
        time = '%.3f'%(df['avg_time'].tolist()[0])
        
        gsens_list = np.array(eval(str(df['gsens_list'].tolist()[0])))
        gsens = np.median(gsens_list)
        
        bias_ans_list = np.array(eval(str(df['bias_ans_list'].tolist()[0])))
        bias_ans_list = np.abs(bias_ans_list - true_ans) / true_ans
        #bias = np.mean(bias_ans_list)
        bias = np.median(bias_ans_list)
        bias = '%.2f%%'%(bias*100)
        #print(bias)
        
        nosy_ans_list = np.array([max(i,0) for i in eval(str(df['nosy_ans_list'].tolist()[0]))])
        err_list = np.abs(nosy_ans_list - true_ans) / true_ans
        #error = np.mean(err_list)
        error = np.median(err_list)
        error = '%.2f%%'%(error*100)
        
        
        #print(error)
        
        cols += [bias, bf(gsens), error, time]
    cols = [q, q_descp[q], true_ans] + cols
    print(*cols, sep='\t')

query	type	true answer	bias	sensitivity	error	time	bias	sensitivity	error	time
q1	path join	60175	3.44%	119	3.56%	0.693	1.02%	220	1.34%	0.292
q2	acyclic join	60175	7.62%	640	7.71%	0.554	100.00%	774	99.03%	0.231
q3	cyclic join	2333	0.00%	 14	2.84%	23.063	2.14%	12375000	1293.53%	0.546
